<a href="https://colab.research.google.com/github/BeataStultica/lstm_labs/blob/main/laba7_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import re
import json
import pandas as pd

In [2]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
#df = pd.read_json('/content/gdrive/MyDrive/yelp_academic_dataset_review.json', lines=True)
#df.head
data = []
column = ['stars', 'text']
with open('/content/gdrive/MyDrive/yelp_academic_dataset_review.json', encoding='latin-1') as f:
  for line in f:
    st = json.loads(line)
    out_line = [st['text'], st['stars']]
    data.append(out_line)


In [4]:
df = pd.DataFrame(data=data, columns= column)
df.columns = ['text', 'stars']
df.head

<bound method NDFrame.head of                                                       text  stars
0        If you decide to eat here, just be aware it is...    3.0
1        I've taken a lot of spin classes over the year...    5.0
2        Family diner. Had the buffet. Eclectic assortm...    3.0
3        Wow!  Yummy, different,  delicious.   Our favo...    5.0
4        Cute interior and owner (?) gave us tour of up...    4.0
...                                                    ...    ...
6990275  Latest addition to services from ICCU is Apple...    5.0
6990276  This spot offers a great, affordable east week...    5.0
6990277  This Home Depot won me over when I needed to g...    4.0
6990278  For when I'm feeling like ignoring my calorie-...    5.0
6990279  Located in the 'Walking District' in Nashville...    3.0

[6990280 rows x 2 columns]>

In [10]:
data = df[['text', 'stars']][:1000000]
embed_dim = 128
lstm_out = 256
batch_size= 128
n_epoch = 5
word_count = 20000

data['sentiment']=[1 if (x>3) else 0 for x in data['stars']]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
#for idx,row in data.iterrows():
#    row[0] = row[0].replace('rt',' ')
#data['text']= [x.encode('ascii') for x in data['text']]

tokenizer = Tokenizer(nb_words = word_count, split=' ')
tokenizer.fit_on_texts(data['text'].values)

X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X, maxlen=50)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [11]:
data.head

<bound method NDFrame.head of                                                      text  stars  sentiment
0       if you decide to eat here just be aware it is ...    3.0          0
1       ive taken a lot of spin classes over the years...    5.0          1
2       family diner had the buffet eclectic assortmen...    3.0          0
3       wow  yummy different  delicious   our favorite...    5.0          1
4       cute interior and owner  gave us tour of upcom...    4.0          1
...                                                   ...    ...        ...
999995  never really had any issues here other than th...    3.0          0
999996  fish recently moved a couple of doors down the...    4.0          1
999997  ive been to south house around a dozen or so t...    3.0          0
999998  wow i am shocked at these reviews i have tried...    1.0          0
999999  headed to st louis for a little getaway with m...    4.0          1

[1000000 rows x 3 columns]>

In [12]:



model = Sequential()
model.add(Embedding(20000, 100, input_length = 50))#word_count, embed_dim,input_length = 50))#X.shape[1]))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])

Y = data['sentiment']#pd.get_dummies(data['sentiment']).values
X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y, test_size = 0.20, random_state = 21)

In [13]:
from keras.callbacks import ModelCheckpoint
model_path = '/content/gdrive/MyDrive/last_model_lab7.h5'
callback = ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True)
model.fit(X_train, Y_train, batch_size =batch_size, epochs = n_epoch, validation_data=(X_valid, Y_valid), 
          steps_per_epoch=len(X_train)//batch_size, validation_steps=len(X_valid)//batch_size, callbacks = [callback])


#score,acc = model.evaluate(X_valid, Y_valid, batch_size = batch_size)
#print("Logloss score: %.2f" % (score))
#print("Validation set Accuracy: %.2f" % (acc))

Epoch 1/5
6250/6250 [==============================] - 603s 96ms/step - loss: 0.2550 - accuracy: 0.8951 - val_loss: 0.2293 - val_accuracy: 0.9063
Epoch 2/5
6250/6250 [==============================] - 621s 99ms/step - loss: 0.2106 - accuracy: 0.9154 - val_loss: 0.2225 - val_accuracy: 0.9109
Epoch 3/5
6250/6250 [==============================] - 605s 97ms/step - loss: 0.1908 - accuracy: 0.9241 - val_loss: 0.2200 - val_accuracy: 0.9123
Epoch 4/5
6250/6250 [==============================] - 587s 94ms/step - loss: 0.1737 - accuracy: 0.9311 - val_loss: 0.2261 - val_accuracy: 0.9120
Epoch 5/5
6250/6250 [==============================] - 583s 93ms/step - loss: 0.1584 - accuracy: 0.9377 - val_loss: 0.2413 - val_accuracy: 0.9100


In [17]:
from keras.models import load_model
model2 = load_model('/content/gdrive/MyDrive/last_model_lab7.h5')
text = ["I hate this service, return my money", "Nice book, i will recommend it to my friends"]
for i in text:
  sequence = tokenizer.texts_to_sequences([i])
  data = pad_sequences(sequence, maxlen=50)
  result = model2.predict(data)
  print(result)
  if result[0][0] <0.5:
    print('negative')
  else:
    print('positive')

[[0.11892862]]
negative
[[0.9699573]]
positive
